In [21]:
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import sys
import argparse
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import json
import pprint
import cv2
import os
import time
import tensorflow as tf
from keras import layers, Input, models, optimizers
from keras.models import Model, Sequential
from keras.layers import MaxPooling2D, Conv2D,Activation, Dropout, Flatten, Dense
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

## 이미지 분리 및 예측 함수

In [65]:
def seperate_n_predict(img_dir, 
                       model_dir_1,
                       model_dir_2,
                       model_dir_3,
                       model_dir_4, var):
    
    conv_base = VGG16(weights = 'imagenet',include_top=False,input_shape=(150,150,3))
    
    model_2 = load_model(model_dir_2)
    model_3 = load_model(model_dir_3)
    model_4 = load_model(model_dir_4)
    
    
    
    # 사용할 model 설정
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_dim = 4 * 4 * 512))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(7, activation='softmax'))
    model.compile(optimizer = optimizers.RMSprop(lr=2e-5),
                  loss='categorical_crossentropy',
                  metrics=['acc'])
    model.load_weights(model_dir)    
    
    
    # 카카오 API 설정
    API_URL = 'https://kapi.kakao.com/v1/vision/product/detect'
    MYAPP_KEY = '7af88d85dc96c6e9ec2ebadca618519a'
    url = "https://kapi.kakao.com/v1/vision/product/detect"       
    headers = {'Authorization': 'KakaoAK {}'.format(MYAPP_KEY)}
    files = { 'file' : open(img_dir, 'rb')}
    response = requests.post(url, headers=headers, files=files)
    
    
    # 카카오 API 저장 및 예외 상황 설정
    info = []
    result = response.json()

    if 'result' in result:                                                    
        fig_w, fig_h = result['result']['width'], result['result']['height']
    else:
        return ['NA','NA','NA','NA','NA','NA','NA']

    if 'objects' in result['result']:
        mylist = result['result']['objects']
        cnt = 0
        for objects in mylist:
            if objects['class'] in ['shirts','blouse','t-shirts','pants','outer','skirt','one-piece']:
                cnt = cnt+1
        if cnt > 4:
            return ['NA','NA','NA','NA','NA','NA','NA']
    else:
        return ['NA','NA','NA','NA','NA','NA','NA']
        
        
    for each in result['result']['objects']:                        
            each = list(each.values())                                            
            info.append(each)
            
    
    
    upper = list()
    lower = list()
    for _ in result['result']['objects']:
        if _['class'] in ('t-shirts', 'shirts'):
            upper = _
        elif _['class'] in ('skirt', 'pants'):
            lower = _
            
    x1 = lower['x1']*fig_w -5
    y1 = lower['y1']*fig_h -5
    w1 = lower['x2']*fig_w +5
    h1 = lower['y2']*fig_h +5

    x2 = upper['x1']*fig_w -5
    y2 = upper['y1']*fig_h -5
    w2 = upper['x2']*fig_w +5
    h2 = y1
        
    
    top_1 = []    
    bottom_1 = []
    top_2 = []    
    bottom_2 = []    
    top_3 = []    
    bottom_3 = []    
    top_4 = []    
    bottom_4 = []    
    
    
    for _ in range(len(info)):
        img = Image.open(img_dir)
        category = info[_][4]
        
        if category in ('shirts','t-shirts'):

            img = img.crop((x2,y2,w2,h2))                               
            img.show() 
      
            # 4) 자른 사진 model 예측 
            img = img.resize((150,150))
            data = np.array(img)
            data = data.astype('float')
            data = data/255
            data = data.reshape((1,150,150,3))
            
            p = conv_base.predict(data)
            p = p.reshape((-1,8192))
            p = model.predict_classes(p)[0]
            p = var[p]
            top_1.append(p)
        
            result_2 = var[model_2.predict_classes(data)[0]]
            top_2.append(result_2)
            
            result_3 = var[model_3.predict_classes(data)[0]]
            top_3.append(result_3)
        
            result_4 = var[model_4.predict_classes(data)[0]]
            top_4.append(result_4)        
        
        elif category in ('pants','skirt'): 
            img = img.crop((x1,y1,w1,h1))                               
            img.show() 
            
            # 4) 자른 사진 model 예측 
            img = img.resize((150,150))
            data = np.array(img)
            data = data.astype('float')
            data = data/255
            data = data.reshape((1,150,150,3))

            p = conv_base.predict(data)
            p = p.reshape((-1,8192))
            p = model.predict_classes(p)[0]
            p = var[p]
            bottom_1.append(p)
            
            result_2 = var[model_2.predict_classes(data)[0]]
            bottom_2.append(result_2)
            
            result_3 = var[model_3.predict_classes(data)[0]]
            bottom_3.append(result_3)
        
            result_4 = var[model_4.predict_classes(data)[0]]
            bottom_4.append(result_4)        
        
            
    return (f'model_1 :{top_1, bottom_1} \n model_2 :{top_2, bottom_2} \n model_3 :{top_3, bottom_3}\n model_4 :{top_4, bottom_4}')


## 여러 이미지 상하의 pattern 예측 및 값 저장 함수

In [ ]:
# 하나의 폴더에 있는 여러 이미지를 불러와서 상하의 crop 하고 pattern 예측한 값을
# 돌려받아 이를 dataframe 으로 append 및 하나의 파일로 저장하는 함수 구현

In [66]:
def predict_multi(folder_dir, model_dir, var):
    
    file_list = os.listdir(folder_dir)
    
    for _ in range(len(file_list)):
        img_dir = folder_dir + '/' + file_list[_]
        Image.open(img_dir)
        k = seperate_n_predict(img_dir, 
                               model_dir_1,
                               model_dir_2,
                               model_dir_3,
                               model_dir_4,
                               var)
        print(f'========{file_list[_]}==========')
        print(k)
   

    return 


In [67]:
var = ['check', 'dot', 'floral', 'graphic', 'leopard', 'none', 'stripe']
folder_dir = './img/test'    
model_dir_1 = './model/pattern_9985.h5'   
model_dir_2 = './model/pattern_7class_정확도77.h5'
model_dir_3 = './model/pattern_new7class_6578.h5'
model_dir_4 = './model/pattern_no_camo_정확도76.h5'

In [68]:
predict_multi(folder_dir, model_dir, var)

========Aa (10).jpg==========
model_1 :(['graphic'], ['check']) 
 model_2 :(['graphic'], ['none']) 
 model_3 :(['graphic'], ['floral'])
 model_4 :(['graphic'], ['none'])
========Aa (14).jpg==========
model_1 :(['none'], ['check']) 
 model_2 :(['none'], ['check']) 
 model_3 :(['none'], ['check'])
 model_4 :(['none'], ['check'])
========Aa (15).jpg==========
model_1 :(['none'], ['check']) 
 model_2 :(['none'], ['check']) 
 model_3 :(['none'], ['floral'])
 model_4 :(['none'], ['none'])
========Aa (16).jpg==========
model_1 :(['graphic'], ['check']) 
 model_2 :(['none'], ['check']) 
 model_3 :(['graphic'], ['check'])
 model_4 :(['graphic'], ['check'])
========Aa (17).jpg==========
model_1 :(['none'], ['check']) 
 model_2 :(['none'], ['check']) 
 model_3 :(['none'], ['floral'])
 model_4 :(['none'], ['none'])
